In [1]:
import pandas as pd
import numpy as np

# chia dữ liệu
from sklearn.model_selection import train_test_split

# tiền xử lý dữ liệu encode, scaler các thứ
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

# các thông số đánh giá mô hình
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import pickle

In [2]:
df =  pd.read_excel("..\\store\\fact.xlsx")

In [3]:
df['chenhlech_khoahoc'] = df['khoahoc'] - df['khoahoc_chuan']

In [4]:
df.columns

Index(['mssv', 'xeploai', 'namsinh', 'noisinh', 'gioitinh', 'khoa', 'hedt',
       'khoahoc', 'diem_tt', 'dtbhk_hk_1', 'dtbhk_hk_2', 'dtbhk_hk_3',
       'dtbhk_hk_4', 'dtbhk_hk_5', 'sotchk_hk_1', 'sotchk_hk_2', 'sotchk_hk_3',
       'sotchk_hk_4', 'sotchk_hk_5', 'drl_hk_1', 'drl_hk_2', 'drl_hk_3',
       'drl_hk_4', 'drl_hk_5', 'khoahoc_chuan', 'chenhlech_khoahoc'],
      dtype='object')

In [5]:
df['chenhlech_khoahoc'].value_counts()

chenhlech_khoahoc
0     1623
1      126
2       10
3        5
4        4
6        1
11       1
Name: count, dtype: int64

# Định nghĩa hàm để đánh giá

In [6]:
# Tạo results để lưu các thông số đánh giá mô hình
results_df = pd.DataFrame(columns=["Model", "Accuracy", "Precision", "Recall", "F1_score"])

In [7]:
def evaluate_and_save_model(name_model, model, X_test, y_test):
    global results_df
    
    # Dự đoán từ mô hình
    y_pred = model.predict(X_test)
    if len(y_pred.shape) > 1:  # Trường hợp output là xác suất (probability)
        y_pred = y_pred.argmax(axis=1)  # Chọn lớp có xác suất cao nhất
    
    # Tính các chỉ số đánh giá
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')  # Sử dụng weighted nếu có nhiều lớp
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Lưu kết quả vào DataFrame
    new_result = pd.DataFrame({
        "Model": [name_model],
        "Accuracy": [accuracy],
        "Precision": [precision],
        "Recall": [recall],
        "F1_score": [f1],
    })
    
    results_df = pd.concat([results_df, new_result], ignore_index=True)
    return results_df

In [8]:
def evaluate_and_save_model_cnn_2_inputs(name_model, model, X_test_seq, X_test_cat, y_test):
    global results_df
    
    # Dự đoán từ mô hình (CNN với 2 đầu vào)
    y_pred = model.predict([X_test_seq, X_test_cat])
    
    if len(y_pred.shape) > 1:  # Trường hợp output là xác suất (probability)
        y_pred = y_pred.argmax(axis=1)  # Chọn lớp có xác suất cao nhất
    
    # Tính các chỉ số đánh giá
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')  # Sử dụng weighted nếu có nhiều lớp
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Lưu kết quả vào DataFrame
    new_result = pd.DataFrame({
        "Model": [name_model],
        "Accuracy": [accuracy],
        "Precision": [precision],
        "Recall": [recall],
        "F1_score": [f1],
    })
    
    # Cập nhật kết quả vào DataFrame global
    results_df = pd.concat([results_df, new_result], ignore_index=True)
    
    return results_df

In [9]:
df.drop(columns=['mssv', 'namsinh', 'khoahoc'], axis=1, inplace=True)

In [10]:
df.columns

Index(['xeploai', 'noisinh', 'gioitinh', 'khoa', 'hedt', 'diem_tt',
       'dtbhk_hk_1', 'dtbhk_hk_2', 'dtbhk_hk_3', 'dtbhk_hk_4', 'dtbhk_hk_5',
       'sotchk_hk_1', 'sotchk_hk_2', 'sotchk_hk_3', 'sotchk_hk_4',
       'sotchk_hk_5', 'drl_hk_1', 'drl_hk_2', 'drl_hk_3', 'drl_hk_4',
       'drl_hk_5', 'khoahoc_chuan', 'chenhlech_khoahoc'],
      dtype='object')

In [11]:
epochs = 75

# MLP (Multi-Layer Perceptron)

In [12]:
# One-Hot Encoding cho dữ liệu phân loại
mlp_categorical_cols = ["noisinh", "gioitinh", "khoa", "hedt", "chenhlech_khoahoc", "diem_tt"]
mlp_encoder = OneHotEncoder()
mlp_encoded_features = mlp_encoder.fit_transform(df[mlp_categorical_cols]).toarray()

In [13]:
# Chuẩn hóa dữ liệu số
mlp_numerical_cols = [col for col in df.columns if "dtbhk_hk_" in col or "sotchk_hk_" in col or "drl_hk_" in col]
mlp_scaler = MinMaxScaler()
mlp_scaled_features = mlp_scaler.fit_transform(df[mlp_numerical_cols])

In [14]:
# Kết hợp các đặc trưng
mlp_X = np.hstack([mlp_scaled_features, mlp_encoded_features])
# Chuyển xếp loại thành nhãn số
mlp_y = df["xeploai"].astype("category").cat.codes

In [15]:
# Chia dữ liệu
mlp_X_train, mlp_X_temp, mlp_y_train, mlp_y_temp = train_test_split(mlp_X, mlp_y, test_size=0.2, random_state=42)
mlp_X_val, mlp_X_test, mlp_y_val, mlp_y_test = train_test_split(mlp_X_temp, mlp_y_temp, test_size=0.5, random_state=42)

In [16]:
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(mlp_X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(np.unique(mlp_y)), activation='softmax')
])

mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

E:\Anaconda 3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# Train model
mlp_history = mlp_model.fit(mlp_X_train, mlp_y_train, validation_data=(mlp_X_val, mlp_y_val), epochs=epochs, batch_size=32)

Epoch 1/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4570 - loss: 1.2063 - val_accuracy: 0.6441 - val_loss: 0.8777
Epoch 2/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5946 - loss: 0.9366 - val_accuracy: 0.6723 - val_loss: 0.8041
Epoch 3/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6330 - loss: 0.8545 - val_accuracy: 0.6780 - val_loss: 0.7359
Epoch 4/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6650 - loss: 0.7938 - val_accuracy: 0.6667 - val_loss: 0.7165
Epoch 5/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7013 - loss: 0.7235 - val_accuracy: 0.6667 - val_loss: 0.6724
Epoch 6/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6807 - loss: 0.6939 - val_accuracy: 0.6667 - val_loss: 0.6522
Epoch 7/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7180 - loss: 0.6683 - val_accuracy: 0.6610 - val_loss: 0.6320
Epoch 8/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7239 - loss: 0.6146 - val_accuracy: 0.6723 - val_loss:

In [18]:
evaluate_and_save_model("MLP - Multi-Layer Perceptron", mlp_model, mlp_X_test, mlp_y_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


E:\Anaconda 3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\anhthy\AppData\Local\Temp\ipykernel_3044\2568259362.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_result], ignore_index=True)


,Model,Accuracy,Precision,Recall,F1_score
0,MLP - Multi-Layer Perceptron,0.745763,0.728814,0.745763,0.728029


# LSTM (Long Short-Term Memory)

In [19]:
# LSTM phù hợp khi phân tích dữ liệu tuần tự, như điểm trung bình, số tín chỉ, và điểm rèn luyện qua từng học kỳ 
# vì vậy có thể chuyển dổi điểm trung bình học kỳ, số tín chỉ, điểm rèn luyện sang các list tuần tự

In [20]:
# Dữ liệu chuỗi (chỉ chọn các cột tuần tự)
lstm_seq_features = [f"dtbhk_hk_{i}" for i in range(1, 6)] + [f"sotchk_hk_{i}" for i in range(1, 6)] + [f"drl_hk_{i}" for i in range(1, 6)]
# Đổi sang numpy cho phù hợp với LSTM model
lstm_df_numpy = df[lstm_seq_features].values
# Reshape: (samples, timesteps, features)
lstm_sequence_data = lstm_df_numpy.reshape(lstm_df_numpy.shape[0], 5, -1)

In [21]:
# Chuẩn hóa dữ liệu
lstm_scaler = MinMaxScaler()
lstm_sequence_data = lstm_sequence_data.reshape(-1, lstm_sequence_data.shape[-1])
lstm_sequence_data = lstm_scaler.fit_transform(lstm_sequence_data)
lstm_sequence_data = lstm_sequence_data.reshape(-1, 5, len(lstm_seq_features) // 5)

In [22]:
# Dữ liệu label
lstm_y = df["xeploai"].astype("category").cat.codes

In [23]:
# Chia dữ liệu
lstm_X_train, lstm_X_temp, lstm_y_train, lstm_y_temp = train_test_split(lstm_sequence_data, lstm_y, test_size=0.2, random_state=42)
lstm_X_val, lstm_X_test, lstm_y_val, lstm_y_test = train_test_split(lstm_X_temp, lstm_y_temp, test_size=0.5, random_state=42)

In [24]:
# LSTM Model
lstm_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(5, len(lstm_seq_features) // 5), return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(np.unique(lstm_y)), activation='softmax')
])

lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

E:\Anaconda 3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [25]:
lstm_history = lstm_model.fit(lstm_X_train, lstm_y_train, validation_data=(lstm_X_val, lstm_y_val), epochs=epochs, batch_size=32)

Epoch 1/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.4623 - loss: 1.2914 - val_accuracy: 0.6441 - val_loss: 0.9294
Epoch 2/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5980 - loss: 1.0025 - val_accuracy: 0.6441 - val_loss: 0.9032
Epoch 3/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6073 - loss: 0.9661 - val_accuracy: 0.6441 - val_loss: 0.8986
Epoch 4/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5999 - loss: 0.9655 - val_accuracy: 0.6441 - val_loss: 0.8939
Epoch 5/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5986 - loss: 0.9639 - val_accuracy: 0.6441 - val_loss: 0.8941
Epoch 6/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6068 - loss: 0.9612 - val_accuracy: 0.6441 - val_loss: 0.8903
Epoch 7/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6056 - loss: 0.9558 - val_accuracy: 0.6441 - val_loss: 0.8869
Epoch 8/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6084 - loss: 0.9653 - val_accuracy: 0.6441 - val_loss

In [26]:
evaluate_and_save_model("LSTM (Long Short-Term Memory)", lstm_model, lstm_X_test, lstm_y_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step


E:\Anaconda 3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Model,Accuracy,Precision,Recall,F1_score
0,MLP - Multi-Layer Perceptron,0.745763,0.728814,0.745763,0.728029
1,LSTM (Long Short-Term Memory),0.757062,0.743536,0.757062,0.746699


# CNN (Convolutional Neural Network)

In [27]:
# CNN và LSTM dùng chung loại đầu vào dữ liệu tuần tự, nhưng vẫn xử lý lại để để ấn run phát là nó chạy cho nó mượt
# Dữ liệu chuỗi (chỉ chọn các cột tuần tự)
cnn_seq_features = [f"dtbhk_hk_{i}" for i in range(1, 6)] + [f"sotchk_hk_{i}" for i in range(1, 6)] + [f"drl_hk_{i}" for i in range(1, 6)]
cnn_sequence_data = df[cnn_seq_features].values
cnn_sequence_data = cnn_sequence_data.reshape(cnn_sequence_data.shape[0], 5, -1)

In [28]:
# Chuẩn hóa
cnn_scaler = MinMaxScaler()
cnn_sequence_data = cnn_sequence_data.reshape(-1, cnn_sequence_data.shape[-1])
cnn_sequence_data = cnn_scaler.fit_transform(cnn_sequence_data)
cnn_sequence_data = cnn_sequence_data.reshape(-1, 5, len(cnn_seq_features) // 5)

In [29]:
# Dữ liệu label
cnn_y = df["xeploai"].astype("category").cat.codes

In [30]:
# Chia dữ liệu
cnn_X_train, cnn_X_temp, cnn_y_train, cnn_y_temp = train_test_split(cnn_sequence_data, cnn_y, test_size=0.2, random_state=42)
cnn_X_val, cnn_X_test, cnn_y_val, cnn_y_test = train_test_split(cnn_X_temp, cnn_y_temp, test_size=0.5, random_state=42)

In [31]:
# CNN Model
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, kernel_size=2, activation='relu', input_shape=(5, len(cnn_seq_features) // 5)),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(64, kernel_size=2, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(np.unique(cnn_y)), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

E:\Anaconda 3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
cnn_history = cnn_model.fit(cnn_X_train, cnn_y_train, validation_data=(cnn_X_val, cnn_y_val), epochs=epochs, batch_size=32)

Epoch 1/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4755 - loss: 1.2572 - val_accuracy: 0.6441 - val_loss: 0.9466
Epoch 2/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5828 - loss: 1.0547 - val_accuracy: 0.6441 - val_loss: 0.9117
Epoch 3/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5990 - loss: 0.9820 - val_accuracy: 0.6441 - val_loss: 0.9011
Epoch 4/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5998 - loss: 0.9814 - val_accuracy: 0.6441 - val_loss: 0.8952
Epoch 5/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6063 - loss: 0.9672 - val_accuracy: 0.6441 - val_loss: 0.8854
Epoch 6/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6069 - loss: 0.9549 - val_accuracy: 0.6441 - val_loss: 0.8719
Epoch 7/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6200 - loss: 0.9197 - val_accuracy: 0.6441 - val_loss: 0.8446
Epoch 8/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6165 - loss: 0.9005 - val_accuracy: 0.6441 - val_loss:

In [33]:
evaluate_and_save_model("CNN (Convolutional Neural Network)", cnn_model, cnn_X_test, cnn_y_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


E:\Anaconda 3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Model,Accuracy,Precision,Recall,F1_score
0,MLP - Multi-Layer Perceptron,0.745763,0.728814,0.745763,0.728029
1,LSTM (Long Short-Term Memory),0.757062,0.743536,0.757062,0.746699
2,CNN (Convolutional Neural Network),0.819209,0.812835,0.819209,0.815739


# LSTM 2 inputs

In [34]:
# Trích xuất các đặc trưng
lstm_2_inputs_categorical_features = ["noisinh", "gioitinh", "khoa", "hedt", "chenhlech_khoahoc", "diem_tt"]
lstm_2_inputs_sequence_features = [f"dtbhk_hk_{i}" for i in range(1, 6)] + \
                                   [f"sotchk_hk_{i}" for i in range(1, 6)] + \
                                   [f"drl_hk_{i}" for i in range(1, 6)]

In [35]:
# Xử lý các đặc trưng categorical
lstm_2_inputs_cat_data = df[lstm_2_inputs_categorical_features].copy()
for col in lstm_2_inputs_categorical_features:
    le = LabelEncoder()
    lstm_2_inputs_cat_data[col] = le.fit_transform(lstm_2_inputs_cat_data[col])

In [36]:
# Chuẩn hóa dữ liệu categorical
lstm_2_inputs_cat_data = MinMaxScaler().fit_transform(lstm_2_inputs_cat_data)

In [37]:
# Xử lý dữ liệu sequence
lstm_2_inputs_sequence_data = df[lstm_2_inputs_sequence_features].values
lstm_2_inputs_sequence_data = lstm_2_inputs_sequence_data.reshape(lstm_2_inputs_sequence_data.shape[0], 5, -1)
lstm_2_inputs_scaler = MinMaxScaler()
lstm_2_inputs_sequence_data = lstm_2_inputs_sequence_data.reshape(-1, lstm_2_inputs_sequence_data.shape[-1])
lstm_2_inputs_sequence_data = lstm_2_inputs_scaler.fit_transform(lstm_2_inputs_sequence_data)
lstm_2_inputs_sequence_data = lstm_2_inputs_sequence_data.reshape(-1, 5, len(lstm_2_inputs_sequence_features) // 5)

In [38]:
# Chia dữ liệu
lstm_2_inputs_y = df["xeploai"].astype("category").cat.codes
lstm_2_inputs_X_train_seq, lstm_2_inputs_X_temp_seq, lstm_2_inputs_y_train, lstm_2_inputs_y_temp = train_test_split(
    lstm_2_inputs_sequence_data, lstm_2_inputs_y, test_size=0.2, random_state=42)
lstm_2_inputs_X_train_cat, lstm_2_inputs_X_temp_cat = train_test_split(
    lstm_2_inputs_cat_data, test_size=0.2, random_state=42)
lstm_2_inputs_X_val_seq, lstm_2_inputs_X_test_seq, lstm_2_inputs_y_val, lstm_2_inputs_y_test = train_test_split(
    lstm_2_inputs_X_temp_seq, lstm_2_inputs_y_temp, test_size=0.5, random_state=42)
lstm_2_inputs_X_val_cat, lstm_2_inputs_X_test_cat = train_test_split(
    lstm_2_inputs_X_temp_cat, test_size=0.5, random_state=42)

In [39]:
# Tạo lại LSTM với 2 đầu vào model
lstm_2_inputs_sequence_input = tf.keras.Input(shape=(5, len(lstm_2_inputs_sequence_features) // 5))
lstm_2_inputs_x_seq = tf.keras.layers.LSTM(64, return_sequences=True)(lstm_2_inputs_sequence_input)
lstm_2_inputs_x_seq = tf.keras.layers.Dropout(0.3)(lstm_2_inputs_x_seq)
lstm_2_inputs_x_seq = tf.keras.layers.LSTM(32)(lstm_2_inputs_x_seq)
lstm_2_inputs_x_seq = tf.keras.layers.Dropout(0.3)(lstm_2_inputs_x_seq)

# Nhánh xử lý dữ liệu categorical
lstm_2_inputs_categorical_input = tf.keras.Input(shape=(lstm_2_inputs_cat_data.shape[1],))
lstm_2_inputs_x_cat = tf.keras.layers.Dense(64, activation='relu')(lstm_2_inputs_categorical_input)
lstm_2_inputs_x_cat = tf.keras.layers.Dropout(0.3)(lstm_2_inputs_x_cat)

# Kết hợp cả hai nhánh
lstm_2_inputs_combined = tf.keras.layers.concatenate([lstm_2_inputs_x_seq, lstm_2_inputs_x_cat])

# Các tầng Dense cuối cùng để dự đoán
lstm_2_inputs_x = tf.keras.layers.Dense(64, activation='relu')(lstm_2_inputs_combined)
lstm_2_inputs_x = tf.keras.layers.Dropout(0.3)(lstm_2_inputs_x)
lstm_2_inputs_output = tf.keras.layers.Dense(len(np.unique(lstm_2_inputs_y)), activation='softmax')(lstm_2_inputs_x)

lstm_2_inputs_model = tf.keras.Model(
    inputs=[lstm_2_inputs_sequence_input, lstm_2_inputs_categorical_input], 
    outputs=lstm_2_inputs_output)


lstm_2_inputs_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [40]:
lstm_2_inputs_history = lstm_2_inputs_model.fit([lstm_2_inputs_X_train_seq, lstm_2_inputs_X_train_cat], lstm_2_inputs_y_train, 
                                                validation_data=([lstm_2_inputs_X_val_seq, lstm_2_inputs_X_val_cat], lstm_2_inputs_y_val), 
                                                epochs=epochs, batch_size=32)

evaluate_and_save_model_cnn_2_inputs("LSTM (Long Short-Term Memory) 2 Inputs", lstm_2_inputs_model, lstm_2_inputs_X_test_seq, lstm_2_inputs_X_test_cat, lstm_2_inputs_y_test)

Epoch 1/75


E:\Anaconda 3\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_20', 'keras_tensor_25']. Received: the structure of inputs=('*', '*')
  warnings.warn(


45/45 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - accuracy: 0.5008 - loss: 1.2166 - val_accuracy: 0.6441 - val_loss: 0.9283
Epoch 2/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6120 - loss: 0.9991 - val_accuracy: 0.6441 - val_loss: 0.9024
Epoch 3/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5991 - loss: 0.9782 - val_accuracy: 0.6441 - val_loss: 0.9001
Epoch 4/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6248 - loss: 0.9471 - val_accuracy: 0.6441 - val_loss: 0.9081
Epoch 5/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6182 - loss: 0.9283 - val_accuracy: 0.6441 - val_loss: 0.9129
Epoch 6/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6152 - loss: 0.9513 - val_accuracy: 0.6441 - val_loss: 0.8935
Epoch 7/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6079 - loss: 0.9504 - val_accuracy: 0.6441 - val_loss: 0.8858
Epoch 8/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6318 - loss: 0.9081 - val_accuracy: 0.6441 - val_loss: 0.8750
Ep

E:\Anaconda 3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Model,Accuracy,Precision,Recall,F1_score
0,MLP - Multi-Layer Perceptron,0.745763,0.728814,0.745763,0.728029
1,LSTM (Long Short-Term Memory),0.757062,0.743536,0.757062,0.746699
2,CNN (Convolutional Neural Network),0.819209,0.812835,0.819209,0.815739
3,LSTM (Long Short-Term Memory) 2 Inputs,0.683616,0.706980,0.683616,0.669734


# CNN 2 inputs

In [41]:
# Tạo label mapping để sử dụng khi deploy model
label_mapping = dict(enumerate(df["xeploai"].astype("category").cat.categories))

label_mapping

# Lưu label_mapping vào file pickle
with open("..\\store\\pickle\\label_mapping.pkl", "wb") as f:
    pickle.dump(label_mapping, f)

In [42]:
# trích xuất các features bao gồm cả categorical và sequences
cnn_2_inputs_categorical_features = ["noisinh", "gioitinh", "khoa", "hedt", "chenhlech_khoahoc", "diem_tt"]
cnn_2_inputs_sequence_features = [f"dtbhk_hk_{i}" for i in range(1, 6)] + \
                                  [f"sotchk_hk_{i}" for i in range(1, 6)] + \
                                  [f"drl_hk_{i}" for i in range(1, 6)]

In [43]:
# Xử lý các đặc trưng categorical
cnn_2_inputs_cat_data = df[cnn_2_inputs_categorical_features].copy()
cnn_2_inputs_label_encoders = {}
for col in cnn_2_inputs_categorical_features:
    le = LabelEncoder()
    cnn_2_inputs_cat_data[col] = le.fit_transform(cnn_2_inputs_cat_data[col])
    cnn_2_inputs_label_encoders[col] = le

# Lưu các LabelEncoders
with open("..\\store\\pickle\\label_encoders.pkl", "wb") as f:
    pickle.dump(cnn_2_inputs_label_encoders, f)

In [44]:
# Chuẩn hóa categorical data
cnn_2_inputs_cat_data = MinMaxScaler().fit_transform(cnn_2_inputs_cat_data)

In [45]:
cnn_2_inputs_label_encoders

{'noisinh': LabelEncoder(),
 'gioitinh': LabelEncoder(),
 'khoa': LabelEncoder(),
 'hedt': LabelEncoder(),
 'chenhlech_khoahoc': LabelEncoder(),
 'diem_tt': LabelEncoder()}

In [46]:
# Huấn luyện scaler_cat
cnn_2_inputs_scaler_cat = MinMaxScaler()
cnn_2_inputs_scaler_cat.fit(cnn_2_inputs_cat_data)

# Lưu scaler_cat vào file
with open("..\\store\\pickle\\scaler_cat.pkl", "wb") as f:
    pickle.dump(cnn_2_inputs_scaler_cat, f)

In [47]:
cnn_2_inputs_sequence_data = df[cnn_2_inputs_sequence_features].values
cnn_2_inputs_sequence_data = cnn_2_inputs_sequence_data.reshape(
    cnn_2_inputs_sequence_data.shape[0], 5, -1)  # (n_samples, 5, n_features_per_step)

In [48]:
# Chuyển sequence_data thành 2D để fit MinMaxScaler
cnn_2_inputs_scaler_seq = MinMaxScaler()
cnn_2_inputs_sequence_data_2d = cnn_2_inputs_sequence_data.reshape(
    -1, cnn_2_inputs_sequence_data.shape[-1])  # (n_samples * 5, n_features_per_step)

In [49]:
# Huấn luyện scaler
cnn_2_inputs_scaler_seq.fit(cnn_2_inputs_sequence_data_2d)

# Lưu scaler đã huấn luyện
with open("..\\store\\pickle\\scaler_seq.pkl", "wb") as f:
    pickle.dump(cnn_2_inputs_scaler_seq, f)

In [50]:
# Chuẩn hóa sequence_data và chuyển lại về 3D
cnn_2_inputs_sequence_data_2d_scaled = cnn_2_inputs_scaler_seq.transform(
    cnn_2_inputs_sequence_data_2d)  # Chuẩn hóa dữ liệu 2D
cnn_2_inputs_sequence_data_scaled = cnn_2_inputs_sequence_data_2d_scaled.reshape(
    -1, 5, cnn_2_inputs_sequence_data.shape[-1])  # Chuyển về 3D

In [51]:
# Chia dữ liệu
cnn_2_inputs_y = df["xeploai"].astype("category").cat.codes
cnn_2_inputs_X_train_seq, cnn_2_inputs_X_temp_seq, cnn_2_inputs_y_train, cnn_2_inputs_y_temp = train_test_split(
    cnn_2_inputs_sequence_data_scaled, cnn_2_inputs_y, test_size=0.2, random_state=42)
cnn_2_inputs_X_train_cat, cnn_2_inputs_X_temp_cat = train_test_split(
    cnn_2_inputs_cat_data, test_size=0.2, random_state=42)
cnn_2_inputs_X_val_seq, cnn_2_inputs_X_test_seq, cnn_2_inputs_y_val, cnn_2_inputs_y_test = train_test_split(
    cnn_2_inputs_X_temp_seq, cnn_2_inputs_y_temp, test_size=0.5, random_state=42)
cnn_2_inputs_X_val_cat, cnn_2_inputs_X_test_cat = train_test_split(
    cnn_2_inputs_X_temp_cat, test_size=0.5, random_state=42)

In [52]:
# Tạo lại CNN với 2 đầu vào model
cnn_2_inputs_sequence_input = tf.keras.Input(shape=(5, len(cnn_2_inputs_sequence_features) // 5))
cnn_2_inputs_x_seq = tf.keras.layers.Conv1D(32, kernel_size=2, activation='relu')(cnn_2_inputs_sequence_input)
cnn_2_inputs_x_seq = tf.keras.layers.MaxPooling1D(pool_size=2)(cnn_2_inputs_x_seq)
cnn_2_inputs_x_seq = tf.keras.layers.Conv1D(64, kernel_size=2, activation='relu')(cnn_2_inputs_x_seq)
cnn_2_inputs_x_seq = tf.keras.layers.GlobalMaxPooling1D()(cnn_2_inputs_x_seq)

# Nhánh xử lý categorical
cnn_2_inputs_categorical_input = tf.keras.Input(shape=(cnn_2_inputs_cat_data.shape[1],))
cnn_2_inputs_x_cat = tf.keras.layers.Dense(64, activation='relu')(cnn_2_inputs_categorical_input)

# Kết hợp các nhánh
cnn_2_inputs_combined = tf.keras.layers.concatenate([cnn_2_inputs_x_seq, cnn_2_inputs_x_cat])
cnn_2_inputs_x = tf.keras.layers.Dense(64, activation='relu')(cnn_2_inputs_combined)
cnn_2_inputs_x = tf.keras.layers.Dropout(0.3)(cnn_2_inputs_x)
cnn_2_inputs_output = tf.keras.layers.Dense(len(np.unique(cnn_2_inputs_y)), activation='softmax')(cnn_2_inputs_x)

cnn_2_inputs_model = tf.keras.Model(inputs=[cnn_2_inputs_sequence_input, cnn_2_inputs_categorical_input], 
                                    outputs=cnn_2_inputs_output)

cnn_2_inputs_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [53]:
cnn_2_inputs_history = cnn_2_inputs_model.fit([cnn_2_inputs_X_train_seq, cnn_2_inputs_X_train_cat], cnn_2_inputs_y_train, 
                                              validation_data=([cnn_2_inputs_X_val_seq, cnn_2_inputs_X_val_cat], cnn_2_inputs_y_val), 
                                              epochs=epochs, batch_size=32)

evaluate_and_save_model_cnn_2_inputs("CNN (Convolutional Neural Network) 2 Inputs", cnn_2_inputs_model, cnn_2_inputs_X_test_seq, cnn_2_inputs_X_test_cat, cnn_2_inputs_y_test)

Epoch 1/75


E:\Anaconda 3\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_32', 'keras_tensor_37']. Received: the structure of inputs=('*', '*')
  warnings.warn(


45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5491 - loss: 1.1571 - val_accuracy: 0.6441 - val_loss: 0.9218
Epoch 2/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6139 - loss: 0.9751 - val_accuracy: 0.6441 - val_loss: 0.8982
Epoch 3/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6165 - loss: 0.9496 - val_accuracy: 0.6441 - val_loss: 0.8858
Epoch 4/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6325 - loss: 0.9151 - val_accuracy: 0.6441 - val_loss: 0.8725
Epoch 5/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6181 - loss: 0.9058 - val_accuracy: 0.6441 - val_loss: 0.8450
Epoch 6/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6203 - loss: 0.8742 - val_accuracy: 0.6497 - val_loss: 0.8087
Epoch 7/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6180 - loss: 0.8360 - val_accuracy: 0.6610 - val_loss: 0.7643
Epoch 8/75
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6409 - loss: 0.7998 - val_accuracy: 0.6836 - val_loss: 0.7133
Ep

E:\Anaconda 3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Model,Accuracy,Precision,Recall,F1_score
0,MLP - Multi-Layer Perceptron,0.745763,0.728814,0.745763,0.728029
1,LSTM (Long Short-Term Memory),0.757062,0.743536,0.757062,0.746699
2,CNN (Convolutional Neural Network),0.819209,0.812835,0.819209,0.815739
3,LSTM (Long Short-Term Memory) 2 Inputs,0.683616,0.706980,0.683616,0.669734
4,CNN (Convolutional Neural Network) 2 Inputs,0.841808,0.831991,0.841808,0.835474


# so sánh các mô hình

In [54]:
results_df

,Model,Accuracy,Precision,Recall,F1_score
0,MLP - Multi-Layer Perceptron,0.745763,0.728814,0.745763,0.728029
1,LSTM (Long Short-Term Memory),0.757062,0.743536,0.757062,0.746699
2,CNN (Convolutional Neural Network),0.819209,0.812835,0.819209,0.815739
3,LSTM (Long Short-Term Memory) 2 Inputs,0.683616,0.706980,0.683616,0.669734
4,CNN (Convolutional Neural Network) 2 Inputs,0.841808,0.831991,0.841808,0.835474


# lưu các mô hình

In [55]:
# Lưu mô hình MLP
mlp_model.save("..\\store\\models\\mlp_model.keras")

# Lưu mô hình LSTM
lstm_model.save("..\\store\\models\\lstm_model.keras")
    
# Lưu mô hình CNN
cnn_model.save("..\\store\\models\\cnn_model.keras")

# Lưu mô hình LSTM 2 inputs
lstm_2_inputs_model.save("..\\store\\models\\lstm_model_2_inputs.keras")

# Lưu mô hình CNN 2 inputs
cnn_2_inputs_model.save("..\\store\\models\\cnn_2_inputs_model.keras")